In [1]:
import json
import os
import re
import time
import uuid
import pandas as pd
import requests
from Crypto.Hash import MD5
from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill, Font
from playwright.sync_api import sync_playwright
from loguru import logger
import datetime
import arrow
import shutil
import openpyxl as op
import numpy as np

In [2]:
df = pd.read_excel(r'C:\Users\EDY\Desktop\jieli_test.xlsx',header=0)
print(df.columns)
print(len(df.columns))

Index(['查询时间', 'DP单号', '提单号', '快递单号', '产品类型', '末端代理类型', '代理', '①德邦外发交接',
       'Unnamed: 8', '②UPS/DHL/FedEx提取', 'Unnamed: 10', '③起飞/开船',
       'Unnamed: 12', '④抵达/到港', 'Unnamed: 14', '⑤目的国海关放行', 'Unnamed: 16',
       '⑥快递派送', 'Unnamed: 18', '⑦快递签收', 'Unnamed: 20', '跟踪节点1 -提取-三大',
       'Unnamed: 22', '跟踪节点1 -起飞-专线', 'Unnamed: 24', '跟踪节点2-进口清关-三大',
       'Unnamed: 26', '跟踪节点2-进口清关-专线', 'Unnamed: 28', '跟踪节点3-签收',
       'Unnamed: 30', '代理整体时效', 'Unnamed: 32', '是否有轨迹异常字段', '异常字段描述',
       '节点时效是否异常'],
      dtype='object')
36


In [79]:
data_dict = {"查询时间": [None, None], "DP单号": [None, None], "提单号": [None, None], "快递单号": [None, None],
             "产品类型": [None, None], "末端代理类型": [None, None], "代理": [None, None], "①德邦外发交接": [None, None],
             "Unnamed: 8": [None, None], "②UPS/DHL/FedEx提取": [None, None], "Unnamed: 10": [None, None],
             "③起飞/开船": [None, None], "Unnamed: 12": [None, None], "④抵达/到港": [None, None], "Unnamed: 14": [None, None],
             "⑤目的国海关放行": [None, None], "Unnamed: 16": [None, None], "⑥快递派送": [None, None], "Unnamed: 18": [None, None],
             "⑦快递签收": [None, None], "Unnamed: 20": [None, None], "跟踪节点1 -提取-三大": [None, None], "Unnamed: 22": [None, None],
             "跟踪节点1 -起飞-专线": [None, None], "Unnamed: 24": [None, None], "跟踪节点2-进口清关-三大": [None, None],
             "Unnamed: 26": [None, None], "跟踪节点2-进口清关-专线": [None, None], "Unnamed: 28": [None, None],
             "跟踪节点3-签收": [None, None], "Unnamed: 30": [None, None], "代理整体时效": [None, None], "Unnamed: 32": [None, None],
             "是否有轨迹异常字段": [None, None],"异常字段描述": [None, None],"节点时效是否异常": [None, None],"异常时效描述": [None, None]}
print(len(data_dict))
data_check = {"国际特惠-上海-日本": [7, 3, 2, 2], "国际特惠-上海-韩国": [7, 3, 2, 2], "台湾海快-厦门-厦门-台湾-普通": [10, 3, 4, 3],
     "台湾海快-厦门-厦门-台湾-移仓": [13, 3, 7, 3], "台湾海快-厦门-厦门-台湾-正报": [13, 3, 7, 3], "港澳台特惠-上海-台湾": [4, 1, 1, 2],
     "台湾海快-上海-台湾": [8, 4, 1, 3], "港澳台特惠-深圳-台湾": [5, 1, 2, 2], "国际特惠-上海/北京/青岛-韩国": [6, 2, 2, 2],
     "国际特惠-上海/北京/青岛-日本": [6, 2, 2, 2], "国际海运-深圳-深圳-新加坡": [19, 8, 9, 2], "国际海运-深圳-深圳-马来西亚": [24, 10, 12, 2],
     "国际海运-深圳-深圳-澳大利亚": [37, 12, 18, 7], "国际陆运-深圳-越南": [21, 2, 7, 12], "国际陆运-深圳-柬埔寨": [21, 2, 7, 12],
     "国际陆运-深圳-泰国": [15, 2, 7, 6], "国际特惠-深圳-阿联酋": [10, 2, 3, 5], "国际特惠-深圳-澳大利亚": [12, 5, 3, 4],
     "国际特惠-深圳-马来西亚": [11, 2, 3, 6], "国际特惠-深圳-新加坡": [6, 2, 2, 2], "国际特惠-深圳-中亚": [10, 2, 3, 5],
     "国际特惠-水晶专业市场-深圳-马来西亚": [11, 2, 3, 6], "国际特惠-水晶专业市场-深圳-新加坡": [5, 2, 1, 2], "四会玉石市场--专线-深圳-新加坡": [6, 2, 2, 2],
     "四会玉石市场--专线-深圳-马来西亚": [11, 2, 3, 6], "四会玉石市场--专线-深圳-越南": [21, 2, 7, 12], "四会玉石市场--专线-深圳-阿联酋": [10, 2, 3, 5],
     "四会玉石市场--专线-深圳-柬埔寨": [21, 2, 7, 12],  "国际标快-上海/北京/青岛-澳大利亚": [8, 3, 3, 2],
     "国际标快-上海/北京/青岛-菲律宾": [6, 2, 1, 2], "国际标快-上海/北京/青岛-韩国": [5, 2, 1, 2], "国际标快-上海/北京/青岛-加拿大": [6, 2, 3, 1],
     "国际标快-上海/北京/青岛-柬埔寨": [6, 2, 2, 2], "国际标快-上海/北京/青岛-马来西亚": [6, 2, 2, 2], "国际标快-上海/北京/青岛-美国": [5, 2, 2, 1],
     "国际标快-上海/北京/青岛-墨西哥": [5, 1, 3, 1], "国际标快-上海/北京/青岛-欧洲": [6, 2, 2, 2], "国际标快-上海/北京/青岛-爱尔兰": [6, 2, 2, 2],
     "国际标快-上海/北京/青岛-瑞士": [6, 2, 2, 2], "国际标快-上海/北京/青岛-荷兰": [6, 2, 2, 2], "国际标快-上海/北京/青岛-意大利": [6, 2, 2, 2],
     "国际标快-上海/北京/青岛-法国": [6, 2, 2, 2], "国际标快-上海/北京/青岛-德国": [6, 2, 2, 2], "国际标快-上海/北京/青岛-英国": [6, 2, 2, 2],
     "国际标快-上海/北京/青岛-其他": [6, 1, 4, 1], "国际标快-上海/北京/青岛-日本": [5, 1, 2, 1], "国际标快-上海/北京/青岛-泰国": [5, 2, 1, 2],
     "国际标快-上海/北京/青岛-新加坡": [5, 2, 1, 2], "国际标快-上海/北京/青岛-新西兰": [5, 1, 3, 1], "国际标快-上海/北京/青岛-印度": [5, 1, 3, 1],
     "国际标快-上海/北京/青岛-东南亚": [6, 2, 2, 2], "国际标快-上海/北京/青岛-印度尼西亚": [6, 2, 2, 2], "国际标快-上海/北京/青岛-越南": [6, 2, 2, 2],
     "国际特惠-上海/北京/青岛-加拿大": [7, 2, 3, 2], "国际特惠-上海/北京/青岛-美国": [6, 2, 3, 1], "国际特惠-上海/北京/青岛-墨西哥": [6, 1, 4, 1],
     "国际特惠-上海/北京/青岛-欧洲": [5, 1, 3, 1], "国际特惠-上海/北京/青岛-其他": [7, 1, 5, 1], "国际标快-上海/北京/青岛-非洲": [7, 1, 5, 1],
     "国际标快-上海/北京/青岛-西亚": [7, 1, 5, 1], "国际标快-深圳-泰国": [6, 2, 2, 2], "国际标快-深圳-新加坡": [6, 2, 2, 2],
     "国际标快-深圳-印度尼西亚": [6, 2, 2, 2], "国际标快-深圳-美国": [7, 2, 2, 3], "国际标快-深圳-马来西亚": [6, 2, 2, 2],
     "国际标快-深圳-越南": [6, 2, 2, 2]}
print(len(data_check))

37
66


In [80]:
df = pd.read_excel(r'C:\Users\EDY\Desktop\jieli_test.xlsx',header=0,dtype=str)
data = df.loc[2:,['查询时间', 'DP单号', '提单号', '快递单号', '产品类型', '末端代理类型', '代理']]
print(data)

   查询时间 DP单号  提单号             快递单号         产品类型 末端代理类型  代理
2     0    1    2       8174088042  港澳台特惠-上海-台湾      5  捷利
3   NaN  NaN  NaN       8174088077  港澳台特惠-上海-台湾    NaN  捷利
4   NaN  NaN  NaN       8174088193   台湾海快-上海-台湾    NaN  捷利
5   NaN  NaN  NaN       8174088236  港澳台特惠-上海-台湾    NaN  捷利
6   NaN  NaN  NaN       8174088261   台湾海快-上海-台湾    NaN  捷利
7   NaN  NaN  NaN       8174088287  港澳台特惠-上海-台湾    NaN  捷利
8   NaN  NaN  NaN       8174088297   台湾海快-上海-台湾    NaN  捷利
9   NaN  NaN  NaN       8174088302  港澳台特惠-上海-台湾    NaN  捷利
10  NaN  NaN  NaN       8174088307   台湾海快-上海-台湾    NaN  捷利
11  NaN  NaN  NaN       8174088320  港澳台特惠-上海-台湾    NaN  捷利
12  NaN  NaN  NaN      88853781461   台湾海快-上海-台湾    NaN  递壹
13  NaN  NaN  NaN      88853781516  港澳台特惠-上海-台湾    NaN  递壹
14  NaN  NaN  NaN      88853781354  港澳台特惠-上海-台湾    NaN  递壹
15  NaN  NaN  NaN      88853781391   台湾海快-上海-台湾    NaN  递壹
16  NaN  NaN  NaN      88853918064  港澳台特惠-上海-台湾    NaN  递壹
17  NaN  NaN  NaN      88853782172   台湾海快-上海-台湾    NaN  

In [86]:
ups_url = r'https://gateway.uniner.com/track-j/api/expressTrack/query'
def str_deal(s1):
    s2 = ' '
    s1 = s1.split()
    s1 = s2.join(s1)
    status = json.loads(s1)
    return status

def time_week(time1='2021-10-12',time2='2021-10-19'):
    time1 = arrow.get(time1).strftime('%Y-%m-%d')
    time1 = datetime.datetime.strptime(time1, '%Y-%m-%d')
    time1_weekday = time1.strftime('%w')
    time2 = arrow.get(time2).strftime('%Y-%m-%d')
    time2 = datetime.datetime.strptime(time2, '%Y-%m-%d')
    time2_weekday = time2.strftime('%w')
    diff = (time2 - time1).days
    week = diff//7
    end = diff%7
    list1 = [int(time1_weekday)]
    for i in range(diff):
        a = list1[-1] + 1
        list1.append(a)
    list2 = [x%7 for x in list1]
    list3 = [x for x in list2 if x in range(1,6)]
    print(time1,time2)
    return diff + 1,len(list3)

def list_change(list1):
    if list1[34]:
        list1[33] = '是'
    else:
        list1[33] = '否'
    if list1[7] is not None and list1[9] is not None:
        day1,day2 = time_week(list1[7], list1[9])
        list1[21] = day1
        list1[22] = day2
    if list1[7] is not None and list1[11] is not None:
        day1,day2 = time_week(list1[7], list1[11])
        list1[23] = day1
        list1[24] = day2
    if list1[9] is not None and list1[15] is not None:
        day1,day2 = time_week(list1[9], list1[15])
        list1[25] = day1
        list1[26] = day2
    if list1[11] is not None and list1[15] is not None:
        day1,day2 = time_week(list1[11], list1[15])
        list1[27] = day1
        list1[28] = day2
    if list1[15] is not None and list1[19] is not None:
        day1,day2 = time_week(list1[15], list1[19])
        list1[29] = day1
        list1[30] = day2
    if list1[7] is not None and list1[19] is not None:
        day1,day2 = time_week(list1[7], list1[19])
        list1[31] = day1
        list1[32] = day2
    if list1[4] in data_check.keys():
        list_check = data_check[list1[4]]
        if list1[22] is not None and list1[22] > list_check[1]:
            list1[36] = list1[36] + '跟踪节点1,'
        elif list1[24] is not None and list1[24] > list_check[1]:
            list1[36] = list1[36] + '跟踪节点1,'
        if list1[26] is not None and list1[26] > list_check[2]:
            list1[36] = list1[36] + '跟踪节点2,'
        elif list1[28] is not None and  list1[28] is not None and list1[28] > list_check[2]:
            list1[36] = list1[36] + '跟踪节点2,'
        if list1[30] is not None and  list1[30] > list_check[3]:
            list1[36] = list1[36] + '跟踪节点3,'
        if list1[32] is not None and list1[32] > list_check[0]:
            list1[36] = list1[36] + '代理整体时效,'
    if list1[36]:
        list1[35] = '是'
    else:
        list1[35] = '否'
    return list1

def get_data_ups(num, list1):
    if str(num) == 'nan':
        return list1
    headers_post = {'Content-Type': 'application/json; charset=utf-8'}
    data_post = {"firmType": 5, "waybillNo": num}  # dpd
    if '捷利' in list1[6]:data_post['firmType'] = 5
    elif '商壹' in list1[6]:data_post['firmType'] = 6   
    elif '台丰' in list1[6]:data_post['firmType'] = 7   
    elif '递壹' in list1[6]:data_post['firmType'] = 8  
    elif '林道' in list1[6]:data_post['firmType'] = 9
    elif '云端' in list1[6]:data_post['firmType'] = 10
    for i in range(3):
        response = requests.post(ups_url, headers=headers_post, json=data_post)
        data_ups = response.json().get('data')
        status = response.json().get('status')
        if status == 500:
            time.sleep(0.95)
            continue
        elif data_ups is None and status == 200:
            return list1
        elif status == 200 and data_post['firmType'] == 5 and '暂时未收到' in response.text:
            time.sleep(0.95)
            continue
        elif data_ups.get('trackNodeVos') is None and status == 200:
            return list1
        else:
            break
    if status == 500:
        return list1
    if data_post['firmType'] == 5:  # 捷利
        for i in data_ups.get('trackNodeVos'):
    #         print(i,'\n')
            Context = str_deal(i['sourceInfo'])['Context']
            time1 = datetime.datetime.strptime(i['trackTime'], '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d')
            if '提取' in i['nodeName']:
                list1[7] = time1
                list1[8] = Context
            elif '起飞/开船' in i['nodeName']:
                list1[11] = time1
                list1[12] = Context
            elif '清关已完成' in Context and list1[15] is None:
                list1[15] = time1
                list1[16] = Context
            elif '派送' in i['nodeName'] and list1[17] is None:
                list1[17] = time1
                list1[18] = Context
            elif '签收' in i['nodeName'] and list1[19] is None:
                list1[19] = time1
                list1[20] = Context
            if '請撥空領取' in Context:
                list1[34] = Context + ',' + list1[34]
            elif '貨件已退回' in Context:
                list1[34] = Context + ',' + list1[34]
            elif '送達客戶不在' in Context:
                list1[34] = Context + ',' + list1[34]
            elif '拒絕收件' in Context:
                list1[34] = Context + ',' + list1[34]
            elif '安检不通过' in Context:
                list1[34] = Context + ',' + list1[34]
    elif data_post['firmType'] == 6:  # 商壹
        for i in data_ups.get('trackNodeVos'):
    #         print(i,'\n')
            Context = str_deal(i['sourceInfo'])['podstatus']
            time1 = datetime.datetime.strptime(i['trackTime'], '%Y-%m-%d%H:%M:%S').strftime('%Y-%m-%d')
            d1 = ['Departure from Origin port', 'Shipment Outbound Forecast', 'Shipment Outbound Scanned']
            if '提取' in i['nodeName']:
                list1[7] = time1
                list1[8] = Context
            elif '起飞/开船' in i['nodeName']:
                if list1[11] is None or 'Departure from Origin port' in Context:
                    list1[11] = time1
                    list1[12] = Context
                elif 'Shipment Outbound Forecast' in Context and 'Shipment Outbound Scanned' in list1[12]:
                    list1[11] = time1
                    list1[12] = Context
            elif 'Cargo Arrived' in Context or 'Arrival at Destination port' in Context:
                list1[13] = time1
                list1[14] = Context
            elif ('Dismantle Cabinets' in Context or 'Shipment Inbound Scanned' in Context or 
                  'Cargo Arrived' in Context) and list1[15] is None:
                list1[15] = time1
                list1[16] = Context
            elif 'out for delivery' in Context or 'Branch Outbound in Delivery Branch' in Context:
                list1[17] = time1
                list1[18] = Context
            elif '签收' in i['nodeName'] and list1[19] is None:
                list1[19] = time1
                list1[20] = Context
        listerror = ['Off-Load','Shipment Delay','拉货','排仓','延误','故障','航次变动','Custom Hold','问题件',
                     'Failed',"Nobody's home",'Problem']
        for n in range(len(listerror)):
            if listerror[n] in response.text:
                list1[34] = list1[34] + listerror[n] + ','
    elif data_post['firmType'] == 7:  # 台丰
        for i in data_ups.get('trackNodeVos'):
    #         print(i,'\n')
            Context = str_deal(i['sourceInfo'])['description']
            time1 = i['trackTime'].split('T')[0]
            d1 = ['Departure from Origin port', 'Shipment Outbound Forecast', 'Shipment Outbound Scanned']
            if '进口报关' in i['nodeName']:
                list1[15] = time1
                list1[16] = Context
            elif '派送' in i['nodeName']:
                list1[17] = time1
                list1[18] = Context
            elif '签收' in i['nodeName'] and list1[19] is None:
                list1[19] = time1
                list1[20] = Context
    elif data_post['firmType'] == 8:  # 递壹时
        for i in data_ups.get('trackNodeVos'):
#             print(i,'\n')
            Context = str_deal(i['sourceInfo'])['status']
            time1 = datetime.datetime.strptime(i['trackTime'], '%Y%m%d').strftime('%Y-%m-%d')
            if '提取' in i['nodeName']:
                list1[7] = time1
                list1[8] = Context
            elif '起飞/开船' in i['nodeName']:
                list1[11] = time1
                list1[12] = Context
            elif '进口报关' in i['nodeName']:
                list1[15] = time1
                list1[16] = Context
            elif '派送' in i['nodeName']:
                list1[17] = time1
                list1[18] = Context
            elif '签收' in i['nodeName'] and list1[19] is None:
                list1[19] = time1
                list1[20] = Context
    elif data_post['firmType'] == 9:  # 林道
        for i in data_ups.get('trackNodeVos'):
#             print(i,'\n')
            Context = str_deal(i['sourceInfo']).split(' ')[2:]
            time1 = i['trackTime']
            if '提取' in i['nodeName']:
                list1[7] = time1
                list1[8] = Context
            elif '起飞/开船' in i['nodeName']:
                list1[11] = time1
                list1[12] = Context
            elif '进口报关' in i['nodeName']:
                list1[15] = time1
                list1[16] = Context
            elif '派送' in i['nodeName']:
                list1[17] = time1
                list1[18] = Context
            elif '签收' in i['nodeName'] and list1[19] is None:
                list1[19] = time1
                list1[20] = Context
            if 'ご不在' in i['sourceInfo']: list1[34] = 'ご不在'
    elif data_post['firmType'] == 10:  # 云端
        for i in data_ups.get('trackNodeVos'):
#             print(i,'\n')
            Context = str_deal(i['sourceInfo'])['详细内容']
            time1 =  datetime.datetime.strptime(i['trackTime'], '%Y-%m-%d %H:%M').strftime('%Y-%m-%d')
            if '提取' in i['nodeName']:
                list1[7] = time1
                list1[8] = Context
            elif '起飞/开船' in i['nodeName']:
                list1[11] = time1
                list1[12] = Context
            elif '进口报关' in i['nodeName']:
                list1[15] = time1
                list1[16] = Context
            elif '派送' in i['nodeName']:
                list1[17] = time1
                list1[18] = Context
            elif '签收' in i['nodeName'] and list1[19] is None:
                list1[19] = time1
                list1[20] = Context
    list1 = list_change(list1)
    return list1

for index, value in data.iterrows():
#     print(index, value['快递单号'])
    list1 = [value['查询时间'],value['DP单号'],value['提单号'],value['快递单号'],value['产品类型'],value['末端代理类型'],value['代理'],
             None,None,None,None,None,None,None,None, None,None,None,None,None,None,None,None,None,None,None,None,None,None,
             None,None,None,None,None,'',None,'']
#     print(list1,'\n')
    list1 = get_data_ups(value['快递单号'],list1)
    print(list1,'\n')

2021-12-13 00:00:00 2021-12-14 00:00:00
2021-12-14 00:00:00 2021-12-15 00:00:00
2021-12-15 00:00:00 2021-12-16 00:00:00
2021-12-13 00:00:00 2021-12-16 00:00:00
['0', '1', '2', '8174088042', '港澳台特惠-上海-台湾', '5', '捷利', '2021-12-13', '已打包，发往下一站', None, None, '2021-12-14', '您的包裹已预配航班', None, None, '2021-12-15', '清关已完成', '2021-12-16', '貨件由屏東營業所人員配送中。', '2021-12-16', '貨件已由屏東營業所送達。貨物件數共1件。', None, None, 2, 2, None, None, 2, 2, 2, 2, 4, 4, '否', '', '是', '跟踪节点1,跟踪节点2,'] 

2021-12-13 00:00:00 2021-12-14 00:00:00
2021-12-14 00:00:00 2021-12-15 00:00:00
2021-12-15 00:00:00 2021-12-16 00:00:00
2021-12-13 00:00:00 2021-12-16 00:00:00
[nan, nan, nan, '8174088077', '港澳台特惠-上海-台湾', nan, '捷利', '2021-12-13', '已打包，发往下一站', None, None, '2021-12-14', '您的包裹已预配航班', None, None, '2021-12-15', '清关已完成', '2021-12-16', '貨件由信義營業所人員配送中。', '2021-12-16', '貨件已由信義營業所送達。貨物件數共1件。', None, None, 2, 2, None, None, 2, 2, 2, 2, 4, 4, '否', '', '是', '跟踪节点1,跟踪节点2,'] 

2021-12-13 00:00:00 2021-12-14 00:00:00
2021-12-14 00:00:00 2021-1

2021-12-04 00:00:00 2021-12-04 00:00:00
2021-12-04 00:00:00 2021-12-07 00:00:00
2021-12-07 00:00:00 2021-12-08 00:00:00
2021-12-04 00:00:00 2021-12-08 00:00:00
[nan, nan, nan, '88853782161', '台湾海快-上海-台湾', nan, '递壹', '2021-12-04', 'Shipment Picked Up|已取件', None, None, '2021-12-04', 'Intransit to Destination|正在中转至目的地', None, None, '2021-12-07', 'Customs Processing|正在清关中', '2021-12-08', 'Shipment Out for Delivery|出外派送', '2021-12-08', 'Shipment Delivered|派送完成', None, None, 1, 0, None, None, 4, 2, 2, 2, 5, 3, '否', '', '是', '跟踪节点2,'] 

2021-12-04 00:00:00 2021-12-04 00:00:00
2021-12-04 00:00:00 2021-12-07 00:00:00
2021-12-07 00:00:00 2021-12-09 00:00:00
2021-12-04 00:00:00 2021-12-09 00:00:00
[nan, nan, nan, '88853782286', '港澳台特惠-上海-台湾', nan, '递壹', '2021-12-04', 'Shipment Picked Up|已取件', None, None, '2021-12-04', 'Intransit to Destination|正在中转至目的地', None, None, '2021-12-07', 'Customs Processing|正在清关中', '2021-12-09', 'Shipment Scheduled for Delivery|预计派送', '2021-12-09', 'Shipment Delivered|派送

2021-12-10 00:00:00 2021-12-11 00:00:00
2021-12-11 00:00:00 2021-12-12 00:00:00
2021-12-12 00:00:00 2021-12-13 00:00:00
2021-12-10 00:00:00 2021-12-13 00:00:00
[nan, nan, nan, '820107018113', '台湾海快-上海-台湾', nan, '林道', '2021-12-10', ['到达操作中心'], None, None, '2021-12-11', ['离开上海机场口岸'], None, None, '2021/12/12', ['进口清关完成'], None, None, '2021-12-13', ['配達完了'], None, None, 2, 1, None, None, 2, 0, 2, 1, 4, 2, '否', '', '否', ''] 

2021-12-10 00:00:00 2021-12-11 00:00:00
2021-12-11 00:00:00 2021-12-12 00:00:00
2021-12-12 00:00:00 2021-12-14 00:00:00
2021-12-10 00:00:00 2021-12-14 00:00:00
[nan, nan, nan, '820107018102', '港澳台特惠-上海-台湾', nan, '林道', '2021-12-10', ['到达操作中心'], None, None, '2021-12-11', ['离开上海机场口岸'], None, None, '2021/12/12', ['进口清关完成'], None, None, '2021-12-14', ['配達完了'], None, None, 2, 1, None, None, 2, 0, 3, 2, 5, 3, '否', '', '否', ''] 

2021-12-10 00:00:00 2021-12-14 00:00:00
2021-12-10 00:00:00 2021-12-16 00:00:00
[nan, nan, nan, '820107018150', '台湾海快-上海-台湾', nan, '林道', '2021-12-10'

2021-12-01 00:00:00 2021-12-02 00:00:00
[nan, nan, nan, '8174077886', '港澳台特惠-上海-台湾', nan, '台丰', None, None, None, None, None, None, None, None, '2021-12-01', '(149)貨件已到發貨站所', '2021-12-01', '(173)配送中', '2021-12-02', '(102)配送完成', None, None, None, None, None, None, None, None, 2, 2, None, None, '否', '', '否', ''] 

2021-12-01 00:00:00 2021-12-02 00:00:00
[nan, nan, nan, 'DPK210076305319', '台湾海快-上海-台湾', nan, '台丰', None, None, None, None, None, None, None, None, '2021-12-01', '(149)貨件已到發貨站所', '2021-12-02', '(173)配送中', '2021-12-02', '(102)配送完成', None, None, None, None, None, None, None, None, 2, 2, None, None, '否', '', '否', ''] 

2021-12-01 00:00:00 2021-12-02 00:00:00
[nan, nan, nan, 'DPK210133923040', '港澳台特惠-上海-台湾', nan, '台丰', None, None, None, None, None, None, None, None, '2021-12-01', '(149)貨件已到發貨站所', '2021-12-02', '(173)配送中', '2021-12-02', '(102)配送完成', None, None, None, None, None, None, None, None, 2, 2, None, None, '否', '', '否', ''] 

2021-12-01 00:00:00 2021-12-02 00:00:00
[nan, nan

In [ ]:
t = '2022/2/18'
time1 = datetime.datetime.strptime(t,'%Y/%m/%d')
# print(t.strftime('%w'))
# print(t)
p = []
t = time1
for i in range(10):
    t = t+datetime.timedelta(days=1)
    print(t,'是周',t.strftime('%w'))
    if int(t.strftime('%w')) in range(1,6):
        p.append(t)
        
print('p:',len(p),p,'\n')
p = []
p.append('{:.2f}%'.format(2/20000*100))
p
print('当前正在处理第3index条，总共有20000len(单号)条，已处理0.02%')

In [87]:
arrow.get("2021-12-12").strftime('%w')

'0'

In [91]:
list233 = ['9289627675', '2526271495','7352926840','6877669536','3400602251','3400605025','3128553131',
           '3400621335','5159071853','3400597233']
for num in list233:
    headers_post = {'Content-Type': 'application/json; charset=utf-8'}
    data_post = {"firmType": 4, "waybillNo": num}  # dhl
    response = requests.post(ups_url, headers=headers_post, json=data_post)
    data_ups = response.json().get('data')
    status = response.json().get('status')
    print(num,':\n',status,data_ups)
    time.sleep(3)

9289627675 :
 200 {'waybillNo': None, 'trackNodeVos': [{'trackTime': '2021-12-07T14:09:00', 'nodeName': '已派送', 'nodeCode': None, 'location': 'CASABLANCA - MOROCCO', 'sourceInfo': '{"pieceIds":["JD014600009308557172"],"description":"已派送","location":{"address":{"addressLocality":"CASABLANCA - MOROCCO"}},"timestamp":"2021-12-07T14:09:00"}'}, {'trackTime': '2021-12-07T08:58:00', 'nodeName': '快件正在派送中', 'nodeCode': None, 'location': 'CASABLANCA - MOROCCO', 'sourceInfo': '{"pieceIds":["JD014600009308557172"],"description":"快件正在派送中","location":{"address":{"addressLocality":"CASABLANCA - MOROCCO"}},"timestamp":"2021-12-07T08:58:00"}'}, {'trackTime': '2021-12-06T21:51:00', 'nodeName': '到达服务网点 CASABLANCA - MOROCCO', 'nodeCode': None, 'location': 'CASABLANCA - MOROCCO', 'sourceInfo': '{"pieceIds":["JD014600009308557172"],"description":"到达服务网点 CASABLANCA - MOROCCO","location":{"address":{"addressLocality":"CASABLANCA - MOROCCO"}},"timestamp":"2021-12-06T21:51:00"}'}, {'trackTime': '2021-12-06T21:24

6877669536 :
 500 None
3400602251 :
 200 {'waybillNo': None, 'trackNodeVos': [{'trackTime': '2021-12-10T11:03:00', 'nodeName': '已派送', 'nodeCode': None, 'location': 'MAURITIUS - MAURITIUS', 'sourceInfo': '{"pieceIds":["JD014600009308667239"],"description":"已派送","location":{"address":{"addressLocality":"MAURITIUS - MAURITIUS"}},"timestamp":"2021-12-10T11:03:00"}'}, {'trackTime': '2021-12-10T09:54:00', 'nodeName': '快件正在派送中', 'nodeCode': None, 'location': 'MAURITIUS - MAURITIUS', 'sourceInfo': '{"pieceIds":["JD014600009308667239"],"description":"快件正在派送中","location":{"address":{"addressLocality":"MAURITIUS - MAURITIUS"}},"timestamp":"2021-12-10T09:54:00"}'}, {'trackTime': '2021-12-09T22:46:00', 'nodeName': '到达服务网点 MAURITIUS - MAURITIUS', 'nodeCode': None, 'location': 'MAURITIUS - MAURITIUS', 'sourceInfo': '{"pieceIds":["JD014600009308667239"],"description":"到达服务网点 MAURITIUS - MAURITIUS","location":{"address":{"addressLocality":"MAURITIUS - MAURITIUS"}},"timestamp":"2021-12-09T22:46:00"}'}, 

3128553131 :
 200 {'waybillNo': None, 'trackNodeVos': [{'trackTime': '2021-12-09T13:43:00', 'nodeName': '已派送', 'nodeCode': None, 'location': 'BRISBANE - AUSTRALIA', 'sourceInfo': '{"pieceIds":["JD014600009327660116"],"description":"已派送","location":{"address":{"addressLocality":"BRISBANE - AUSTRALIA"}},"timestamp":"2021-12-09T13:43:00"}'}, {'trackTime': '2021-12-09T09:06:00', 'nodeName': '快件正在派送中', 'nodeCode': None, 'location': 'BRISBANE - AUSTRALIA', 'sourceInfo': '{"pieceIds":["JD014600009327660116"],"description":"快件正在派送中","location":{"address":{"addressLocality":"BRISBANE - AUSTRALIA"}},"timestamp":"2021-12-09T09:06:00"}'}, {'trackTime': '2021-12-09T06:40:00', 'nodeName': '海关已放行 BRISBANE - AUSTRALIA', 'nodeCode': None, 'location': 'BRISBANE - AUSTRALIA', 'sourceInfo': '{"pieceIds":["JD014600009327660116"],"description":"海关已放行 BRISBANE - AUSTRALIA","location":{"address":{"addressLocality":"BRISBANE - AUSTRALIA"}},"timestamp":"2021-12-09T06:40:00"}'}, {'trackTime': '2021-12-09T06:37:0

5159071853 :
 200 {'waybillNo': None, 'trackNodeVos': [{'trackTime': '2021-12-11T14:51:00', 'nodeName': '已派送', 'nodeCode': None, 'location': 'ISLAMABAD - PAKISTAN', 'sourceInfo': '{"pieceIds":["JD014600009308788338"],"description":"已派送","location":{"address":{"addressLocality":"ISLAMABAD - PAKISTAN"}},"timestamp":"2021-12-11T14:51:00"}'}, {'trackTime': '2021-12-11T14:51:00', 'nodeName': '快件已完成费用支付', 'nodeCode': None, 'location': 'ISLAMABAD - PAKISTAN', 'sourceInfo': '{"description":"快件已完成费用支付","location":{"address":{"addressLocality":"ISLAMABAD - PAKISTAN"}},"timestamp":"2021-12-11T14:51:00"}'}, {'trackTime': '2021-12-11T14:07:00', 'nodeName': '等待安排派送', 'nodeCode': None, 'location': 'ISLAMABAD - PAKISTAN', 'sourceInfo': '{"pieceIds":["JD014600009308788338"],"description":"等待安排派送","location":{"address":{"addressLocality":"ISLAMABAD - PAKISTAN"}},"timestamp":"2021-12-11T14:07:00"}'}, {'trackTime': '2021-12-11T11:46:00', 'nodeName': '快件正在派送中', 'nodeCode': None, 'location': 'ISLAMABAD - PA

3400597233 :
 500 None
